In [53]:
import pandas as pd

### Carregamento e Tratamento dos dados

In [54]:
dados = pd.read_csv('DB_Teste.csv', sep=';')
dados.sample(5)

,Cliente,ID,Tipo,Data da Venda,Categoria,Vendedor,Regional,Duração do Contrato (Meses),Equipe,Valor,Unnamed: 10
743,Cliente 8,2019-0960,Serviços,11/03/2019,Upselling,Vendedor 3,Brasil,3,Time 1,"R$ 307.112,40",NaN
3,Cliente 10,2018-0168,Serviços,30/04/2018,Upselling,Vendedor 1,Brasil,2,Time 1,"R$ 51.894,86",NaN
703,Cliente 71,2020-2396,Produtos,12/06/2020,Cross selling,Vendedor 7,Brasil,12,Time 5,"R$ 1.000.000,00",NaN
203,Cliente 16,2020-2460,Produtos,30/11/2020,Renovação,Vendedor 20,Brasil,12,Time 3,"R$ 438.000,00",NaN
642,Cliente 52,2019-1886,Serviços,26/11/2019,Upselling,Vendedor 9,Brasil,5,Time 3,"R$ 91.649,60",NaN


Temos aqui uma coluna chamada Unnamed que provavelmente foi criada pelo Pandas, dessa forma, vamos fazer um panorama dos nossos dados

In [55]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 853 entries, 0 to 852
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Cliente                      853 non-null    object 
 1   ID                           853 non-null    object 
 2   Tipo                         853 non-null    object 
 3   Data da Venda                853 non-null    object 
 4   Categoria                    853 non-null    object 
 5   Vendedor                     853 non-null    object 
 6   Regional                     853 non-null    object 
 7   Duração do Contrato (Meses)  853 non-null    int64  
 8   Equipe                       853 non-null    object 
 9   Valor                        853 non-null    object 
 10  Unnamed: 10                  0 non-null      float64
dtypes: float64(1), int64(1), object(9)
memory usage: 73.4+ KB


Vemos que a coluna Unnamed possui todos os valores nulos, então podemos remove-la, pois não precisaremos dela para os objetivos

In [56]:
dados = dados.drop(columns='Unnamed: 10', axis=1)
dados.sample(5)

,Cliente,ID,Tipo,Data da Venda,Categoria,Vendedor,Regional,Duração do Contrato (Meses),Equipe,Valor
470,Cliente 43,2019-1151,Serviços,08/05/2019,Upselling,Vendedor 13,Brasil,12,Time 3,"R$ 343.203,88"
210,Cliente 19,2018-0182,Serviços,23/04/2018,Novo Logo,Vendedor 3,Brasil,1,Time 1,"R$ 6.720,00"
279,Cliente 26,2019-1637,Serviços,24/09/2019,Cross selling,Vendedor 10,Brasil,3,Time 7,"R$ 113.699,64"
588,Cliente 5,2020-2322,Serviços,04/05/2020,Renovação,Vendedor 3,Brasil,3,Time 1,"R$ 262.861,20"
108,Cliente 13,2019-1203,Serviços,04/06/2019,Upselling,Vendedor 1,Brasil,3,Time 1,"R$ 133.932,48"


Agora, deve-se verificar se a ID de cada venda são todos únicos, de forma que possamos garantir até certo ponto que os nossos dados são todos únicos.

In [57]:
duplo = dados['ID'].value_counts()
duplo.value_counts()

count
1    851
2      1
Name: count, dtype: int64

Temos aqui 851 ID's únicas, e uma está duplicada, vamos então criar uma máscara para filtrar e listar essas vendas

In [58]:
mask = duplo == 2  #mask recebe True se value_counts de duplo for == 2
duplo_ids = duplo.index[mask] #filtra os ids duplicados a partir de mask True
duplo_ids = list(duplo_ids)
dados.loc[dados['ID'].isin(duplo_ids)].head(10)

,Cliente,ID,Tipo,Data da Venda,Categoria,Vendedor,Regional,Duração do Contrato (Meses),Equipe,Valor
323,Cliente 3,2021-3656,Produtos,07/04/2021,Upselling,Vendedor 28,Brasil,22,Time 1,"R$ 1.305.402,59"
325,Cliente 3,2021-3656,Produtos,23/04/2021,Upselling,Vendedor 28,Brasil,22,Time 1,"R$ 1.345.026,41"


Aqui, numa situação real, deve-se contatar o cliente e verificar se ele sabe o motivo da ID estar duplicada e fazer o tratamento posteriormente disso se for necessário.

Por enquanto, vamos assumir que uma das ID's está incorreta e vamos prosseguir com os restantes dos valores para os nossos objetivos.

Agora, verificando se as datas dos nossos dados correspondem ao que nos foi apresentado (2018 - 2021)

In [59]:
dados['Data da Venda'].min()

'01/03/2018'

In [60]:
dados['Data da Venda'].max()

'31/12/2020'

Por fim, para os nossos objetivos, teremos que utilizar os valores da coluna Valor, mas como ele é do tipo object, precisamos tratar esses dados para um tipo numerico para então prosseguir

Para isso, vamos remover o simbolo do Real e os pontos e alterar o sinal da casa decimal

In [61]:
valor = dados['Valor']
valor = valor.str.replace('R$ ','')
valor = valor.str.replace('.','')
valor = valor.str.replace(',','.')
valor = pd.to_numeric(valor)
valor.info

<bound method Series.info of 0       24800.00
1      302720.00
2       51894.86
3       51894.86
4       51894.86
         ...    
848    273000.00
849    120964.32
850    281880.00
851    720000.00
852    296498.85
Name: Valor, Length: 853, dtype: float64>

Com os valores ajustados, podemos inclui-la nos nossos dados e partir para os objetivos

In [62]:
dados = pd.concat((dados, valor.rename('Valores')), axis=1)
dados.sample(5)

,Cliente,ID,Tipo,Data da Venda,Categoria,Vendedor,Regional,Duração do Contrato (Meses),Equipe,Valor,Valores
614,Cliente 5,2021-3497,Serviços,11/02/2021,Upselling,Vendedor 3,Brasil,12,Time 1,"R$ 359.818,20",359818.20
786,Cliente 8,2020-3157,Serviços,12/01/2021,Renovação,Vendedor 3,Brasil,12,Time 1,"R$ 342.720,00",342720.00
440,Cliente 4,2020-2922,Licenciamento,15/10/2020,Upselling,Vendedor 3,Brasil,25,Time 1,"R$ 333.238,92",333238.92
690,Cliente 7,2018-0033,Licenciamento,31/01/2018,Novo Logo,Vendedor 1,Brasil,37,Time 1,"R$ 338.365,31",338365.31
797,Cliente 81,2020-2638,Produtos,27/08/2020,Cross selling,Vendedor 16,Brasil,12,Time 3,"R$ 158.231,04",158231.04


### Construa uma tabela auxiliar que sumarize o valor vendido por cada vendedor, ordenando do maior para o menor;

In [63]:
vendedores = dados[['Vendedor','Valores']].groupby(by=['Vendedor']).sum().reset_index()
vendedores = vendedores.rename(columns={'Valores' : 'Valor Vendido'})
vendedores = vendedores.sort_values(by='Valor Vendido', ascending=False)
vendedores.head(10)

,Vendedor,Valor Vendido
22,Vendedor 3,1.662205e+08
27,Vendedor 7,5.779918e+07
1,Vendedor 10,5.221206e+07
28,Vendedor 8,4.727451e+07
0,Vendedor 1,3.092948e+07
29,Vendedor 9,2.637456e+07
4,Vendedor 13,2.483730e+07
9,Vendedor 18,2.266230e+07
21,Vendedor 29,1.907687e+07
24,Vendedor 4,1.734682e+07


### Imprima e identifica qual foi o cliente responsável pela venda com maior valor e com menor valor;

In [64]:
clienteMax = dados.loc[dados['Valores'].idxmax()]
print('O cliente com o maior valor de venda eh: '+clienteMax['Cliente']+'\nCom o valor de: '+ str(clienteMax['Valores']))

O cliente com o maior valor de venda eh: Cliente 89
Com o valor de: 24970500.0


In [65]:
clienteMin = dados.loc[dados['Valores'].idxmin()]
print('O cliente com o menor valor de venda eh: '+clienteMin['Cliente']+'\nCom o valor de: '+ str(clienteMin['Valores']))

O cliente com o menor valor de venda eh: Cliente 120
Com o valor de: 1250.0


### Imprima valor médio por Tipo de venda (Serviços, Licenciamento, Produtos)

In [66]:
tipoVenda = dados[['Tipo','Valores']].groupby(by=['Tipo']).mean().reset_index()
tipoVenda = tipoVenda.rename(columns={'Valores' : 'Media de valor vendido'})
print(tipoVenda)

            Tipo  Media de valor vendido
0  Licenciamento           983448.768963
1       Produtos           893409.190065
2       Serviços           314567.937298


#### Imprima o número de vendas realizada por cliente

In [67]:
print(dados['Cliente'].value_counts())

Cliente
Cliente 5      128
Cliente 8       61
Cliente 4       60
Cliente 10      43
Cliente 3       31
              ... 
Cliente 146      1
Cliente 144      1
Cliente 143      1
Cliente 142      1
Cliente 99       1
Name: count, Length: 157, dtype: int64
